In [3]:

import pymongo
from pymongo import MongoClient
import pandas as pd
import os
import bson


#os.chdir('/Users/shannontran/Downloads') #changing directory to where my files are

In [4]:
# Connecting to Mongo db

In [5]:
client = MongoClient('localhost', 27017)

In [6]:
db = client.proj2_mongo
db

Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'proj2_mongo')

In [ ]:
# Importing datasets

In [7]:
# students.csv
students_df = pd.read_csv('students.csv')
students_df = students_df.iloc[1:] #removes first row with datatypes
students_df.head()

students_df = students_df.astype({'student_id':int, 'total_credits':int}) #changing data types from str to int


In [8]:
# in clubs 
in_club_df = pd.read_csv('in_club.csv')[1:]
in_club_df = in_club_df.astype({'student_id':int, 'club_id':int})

In [9]:
# clubs
club_df = pd.read_csv('club.csv')[1:]
club_df = club_df.astype({'club_id':int, 'num_members':int})

In [10]:
# department
dept_df = pd.read_csv('department.csv')[1:]
dept_df = dept_df.astype({'num_students':int, 'num_staff':int})

In [11]:
# Converting to dict
students_dict = students_df.to_dict('records')
in_club_dict = in_club_df.to_dict('records')
club_dict = club_df.to_dict('records')
dept_dict = dept_df.to_dict('records')

In [12]:
# Nests club into in_club
for i in range(0, len(in_club_dict)):
    club_id = in_club_dict[i]['club_id']
    club_info = club_df[club_df['club_id'] == club_id][['club_name']].iloc[0].to_dict()
    in_club_dict[i]['club_id'] = club_info
    #print(club_info)

In [13]:
# Nests student into in_club
for i in range(0, len(in_club_dict)):
    student_id = in_club_dict[i]['student_id']
    student_info = students_df[students_df['student_id'] == student_id][['first_name', 'last_name']].iloc[0].to_dict()
    in_club_dict[i]['student_id'] = student_info


In [14]:
in_club_dict[1]

{'student_id': {'first_name': 'Forge', 'last_name': 'Launch'},
 'club_id': {'club_name': 'Club Running'},
 'role': 'Member'}

In [15]:
# Connecting to MongoDB

In [16]:
in_club_collection = db.in_club # create collection

In [17]:
in_club_collection.insert_many(in_club_dict) #inserts nested dictionary into in_club collection

In [18]:
# Who are club presidents?
query1 = in_club_collection.find({'role':'President'}, {'_id':0}) # fields set to zero are removed from output (for cleaner look)

In [19]:
import pprint #pretty print
for e in query1:
    pprint.pprint(e)

{'club_id': {'club_name': 'Club Running'},
 'role': 'President',
 'student_id': {'first_name': 'Mary', 'last_name': 'Jane'}}
{'club_id': {'club_name': 'The Math Wizards'},
 'role': 'President',
 'student_id': {'first_name': 'Jon', 'last_name': 'Kashmeeri'}}
{'club_id': {'club_name': 'Computational Data Sciences'},
 'role': 'President',
 'student_id': {'first_name': 'Kennedy', 'last_name': 'Vorkin'}}
{'club_id': {'club_name': 'Club Running'},
 'role': 'President',
 'student_id': {'first_name': 'Mary', 'last_name': 'Jane'}}
{'club_id': {'club_name': 'The Math Wizards'},
 'role': 'President',
 'student_id': {'first_name': 'Jon', 'last_name': 'Kashmeeri'}}
{'club_id': {'club_name': 'Computational Data Sciences'},
 'role': 'President',
 'student_id': {'first_name': 'Kennedy', 'last_name': 'Vorkin'}}
{'club_id': {'club_name': 'Club Running'},
 'role': 'President',
 'student_id': {'first_name': 'Mary', 'last_name': 'Jane'}}
{'club_id': {'club_name': 'The Math Wizards'},
 'role': 'President',


In [20]:
query2 = in_club_collection.find({'role':'Member'}, {'_id':0}) # fields set to zero are removed from output (for cleaner look)

for e in query2:
    pprint.pprint(e)

{'club_id': {'club_name': 'Club Running'},
 'role': 'Member',
 'student_id': {'first_name': 'Forge', 'last_name': 'Launch'}}
{'club_id': {'club_name': 'Club Running'},
 'role': 'Member',
 'student_id': {'first_name': 'Jasmine', 'last_name': 'Wang'}}
{'club_id': {'club_name': 'Alternative Spring Break'},
 'role': 'Member',
 'student_id': {'first_name': 'John', 'last_name': 'Doe'}}
{'club_id': {'club_name': 'The Math Wizards'},
 'role': 'Member',
 'student_id': {'first_name': 'Forge', 'last_name': 'Launch'}}
{'club_id': {'club_name': 'The Math Wizards'},
 'role': 'Member',
 'student_id': {'first_name': 'John', 'last_name': 'Doe'}}
{'club_id': {'club_name': 'The Math Wizards'},
 'role': 'Member',
 'student_id': {'first_name': 'Jose', 'last_name': 'Pein'}}
{'club_id': {'club_name': 'Madison House'},
 'role': 'Member',
 'student_id': {'first_name': 'Mark', 'last_name': 'Maguire'}}
{'club_id': {'club_name': 'Madison House'},
 'role': 'Member',
 'student_id': {'first_name': 'Silas', 'last_nam

In [21]:
# Who are seniors by credits? (>=90 credits)
students_collection = db.students
students_collection.insert_many(students_dict)

In [22]:
query3 = students_collection.find({'total_credits':{'$gte':90}}, {'living_status':0, 'student_id':0, 'dept_code':0}) # fields set to zero are removed from output (for cleaner look)

for e in query3:
    pprint.pprint(e)

{'_id': ObjectId('6297d3832436282fc48606e2'),
 'first_name': 'John',
 'last_name': 'Doe',
 'student_year': 'Senior',
 'total_credits': 100}
{'_id': ObjectId('6297d3832436282fc48606e3'),
 'first_name': 'Jon',
 'last_name': 'Kashmeeri',
 'student_year': 'Senior',
 'total_credits': 113}
{'_id': ObjectId('6297d3832436282fc48606e5'),
 'first_name': 'Mark',
 'last_name': 'Maguire',
 'student_year': 'Senior',
 'total_credits': 95}
{'_id': ObjectId('6297d3832436282fc48606ec'),
 'first_name': 'Silas',
 'last_name': 'French',
 'student_year': 'Senior',
 'total_credits': 90}
{'_id': ObjectId('6297d3832436282fc48606ed'),
 'first_name': 'Ronald',
 'last_name': 'McDonald',
 'student_year': 'Senior',
 'total_credits': 102}
{'_id': ObjectId('6297d3832436282fc48606ee'),
 'first_name': 'Walt',
 'last_name': 'White',
 'student_year': 'Senior',
 'total_credits': 120}
{'_id': ObjectId('6297d3832436282fc48606f1'),
 'first_name': 'Bo',
 'last_name': 'Bo',
 'student_year': 'Senior',
 'total_credits': 92}
{'_i

In [23]:
dept_collection = db.dept
dept_collection.insert_many(dept_dict)

In [26]:
#What are the top 5 most popular majors?
pipe = ([
     { "$sort" : { "num_students": -1 } },
     { "$limit": 5}
   ])

query4 = dept_collection.aggregate(pipe)
for e in query4:
    pprint.pprint(e)

{'_id': ObjectId('6298b49983d88a23300cf0ab'),
 'budget': '18000',
 'dept_code': 'LFIT',
 'dept_name': 'Fitness',
 'num_staff': 16,
 'num_students': 245}
{'_id': ObjectId('629826782436282fc4860723'),
 'budget': '18000',
 'dept_code': 'LFIT',
 'dept_name': 'Fitness',
 'num_staff': 16,
 'num_students': 245}
{'_id': ObjectId('6298b49983d88a23300cf0a3'),
 'budget': '230000',
 'dept_code': 'BIOL',
 'dept_name': 'Biology',
 'num_staff': 108,
 'num_students': 223}
{'_id': ObjectId('629826782436282fc486071b'),
 'budget': '230000',
 'dept_code': 'BIOL',
 'dept_name': 'Biology',
 'num_staff': 108,
 'num_students': 223}
{'_id': ObjectId('6298b49983d88a23300cf0a8'),
 'budget': '16000',
 'dept_code': 'ENGL',
 'dept_name': 'English',
 'num_staff': 24,
 'num_students': 204}


In [324]:
#students_collection.delete_many({})

In [317]:
#in_club_collection.delete_many({})